In [1]:
pip install python-dotenv

You should consider upgrading via the '/home/clovis1er/Documents/code/deblay/token_liquidity_analysis/notebooks/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# System imports
from dotenv import load_dotenv
import os
from pprint import pprint

In [3]:
# CMC imports
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

In [4]:
# Python imports
import json
from datetime import date, timedelta
import pandas as pd
from datetime import datetime, date
from const import symbols

In [5]:
load_dotenv()
CMC_API = os.getenv("CMC_API")

### Trying to get market data with Coinmarket cap :
###### Note : after this trial, I realized that with the free API, it is not possible to access historical data

In [7]:
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/quotes/historical'
parameters = {
    "symbol": "BTC",
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': CMC_API,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  pprint(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  pprint(e)

{'data': {'BTC': {'id': 8796,
                  'is_active': 5291,
                  'is_fiat': None,
                  'name': 'x0zfxo8ips',
                  'quotes': [{'quote': {'USD': {'circulating_supply': 8123,
                                                'market_cap': 0.9751022353095258,
                                                'price': 0.6472848166852312,
                                                'timestamp': '2022-10-19T00:55:21.960Z',
                                                'total_supply': 4481,
                                                'volume_24h': 2085}},
                              'timestamp': '2022-10-19T00:55:21.960Z'},
                             {'quote': {'USD': {'circulating_supply': 7981,
                                                'market_cap': 0.843757330577017,
                                                'price': 0.32168709445058075,
                                                'timestamp': '2022-10-19T00:55:21.960Z'

### Trying with Coingecko API :

In [8]:
def format_date(d):
    return d.strftime("%d-%m-%Y")

url = "https://api.coingecko.com/api/v3/coins/<coin_id>/market_chart?vs_currency=usd&days=<nb_days>"
date_start = date(2022, 1, 1)
today = date.today()
dfs = []

for tiker, id_ in symbols.items():
    full_url = url.replace("<coin_id>", id_).replace("<nb_days>", str((today-date_start).days))
    res = requests.get(full_url)
    if res.status_code == 200:
        volumes = res.json()["total_volumes"]
        new_df = pd.DataFrame(volumes, columns=["Timestamp", tiker])
        new_df[tiker] = new_df[tiker] /1000000
        dfs.append(new_df)
    else:
        print(f"Error with {tiker} : {res.text}")
   

In [9]:
print(dfs)

[         Timestamp           BTC
0    1641081600000  18904.585811
1    1641168000000  33756.128242
2    1641254400000  21890.019532
3    1641340800000  24895.990584
4    1641427200000  31837.641246
..             ...           ...
287  1665878400000  13103.099352
288  1665964800000  15683.330320
289  1666051200000  23713.928578
290  1666137600000  25848.175159
291  1666140886000  25643.151155

[292 rows x 2 columns],          Timestamp         UNI
0    1641081600000  193.649960
1    1641168000000  200.986078
2    1641254400000  349.235836
3    1641340800000  367.529749
4    1641427200000  598.856112
..             ...         ...
287  1665878400000   67.132674
288  1665964800000   55.530513
289  1666051200000   84.220710
290  1666137600000  129.146209
291  1666140869000  131.224730

[292 rows x 2 columns],          Timestamp         XMR
0    1641081600000  246.723262
1    1641168000000  171.457202
2    1641254400000  159.317411
3    1641340800000  145.903590
4    1641427200000  146.03

### Merging DFs into one, with a unified index of dates

In [10]:
crypto_dfs = dfs.copy()
def merge_df(df1, df2):
    return df1.merge(df2, on="Timestamp", how="inner")

def ts_to_date(df):
    return format_date(datetime.fromtimestamp(int(df["Timestamp"])/1000))

for i in range(0, len(crypto_dfs)):
    crypto_dfs[i] = crypto_dfs[i].drop(crypto_dfs[i].tail(1).index)
    crypto_dfs[i]["Timestamp"] = crypto_dfs[i].apply(ts_to_date, axis=1)
    df = crypto_dfs[i] if i == 0 else merge_df(df, crypto_dfs[i])
    
df = df.set_index(['Timestamp'])
print(df)

                     BTC         UNI         XMR         VET          USDT  \
Timestamp                                                                    
02-01-2022  18904.585811  193.649960  246.723262  185.705631  38101.900235   
03-01-2022  33756.128242  200.986078  171.457202  203.457724  35584.063770   
04-01-2022  21890.019532  349.235836  159.317411  271.100955  49571.178475   
05-01-2022  24895.990584  367.529749  145.903590  332.618442  52215.823894   
06-01-2022  31837.641246  598.856112  146.031430  730.545253  66142.943859   
...                  ...         ...         ...         ...           ...   
15-10-2022  34806.651159  146.364642   61.359039   57.806368  41444.709431   
16-10-2022  13103.099352   67.132674   51.267495   46.761801  19333.334086   
17-10-2022  15683.330320   55.530513   49.927776   36.795654  21715.929165   
18-10-2022  23713.928578   84.220710   69.226497   46.537669  30096.016793   
19-10-2022  25848.175159  129.146209  106.811280   58.092571  31

In [11]:
df.to_csv('volumes.csv')

### Getting the mean, and reporting it manually in `choose_coins.ipynb`

In [12]:
df.mean(axis=0)

BTC      27906.919552
UNI        176.651164
XMR        141.861416
VET        218.662144
USDT     52610.055281
BNB       1304.515092
ETH      15839.416048
MATIC      714.773412
DOT        617.998714
DOGE       703.629144
XRP       3664.161923
SAND       441.514805
MANA       332.061829
SUSHI      130.576558
dtype: float64